In [1]:
#### Data
import pandas as pd, numpy as np, time
 
df = []
for i in range(0, 20000):
    vv = (i,  ",".join( [str(t) for t in  np.arange(1000, 2000, 1)  ] ) )
    df.append(vv)
df1 =pd.DataFrame(df, columns = ['id', 'list1'])

df = []
for i in range(0, 20000):
    vv = (i,   ",".join( [str(t) for t in  np.arange(2000, 1500, -1)  ]) )
    df.append(vv)
df2 =pd.DataFrame(df, columns = ['id', 'list2'])  

#### algo
def test_0(df1, df2):
    def rank_adjust2(ll1, ll2,):
        """ Re-rank elements of list1 using ranking of list2"""
        if isinstance(ll1, str): ll1 = ll1.split(",")
        if isinstance(ll2, str): ll2 = ll2.split(",")
        n1, n2 = len(ll1), len(ll2)
        if n2 < 1: return ll1
 
        ll2 = {x:i for i,x in enumerate( ll2 )  }
 
        # log(ll1) ; log(ll2)
 
        adjust, mrank = (1.0 * n1) / n2, n2
        rank3 = np.zeros(n1, dtype='float32')
        kk    = 2
        for rank1, sid in enumerate(ll1):
            rank2        = ll2.get(sid, mrank)
            rank3[rank1] = - rank_score2(rank1, rank2, adjust=adjust, kk= kk)
 
        # Id of ll1 sorted list
        v = [ll1[i] for i in np.argsort(rank3)]
        return ",".join( v)
 
 
    def rank_score2(rank1, rank2, adjust=1.0, kk=1.0):
        return 1.0 / (kk + rank1) + 1.0 / (kk + rank2 * adjust)
 
    dfx1 = df1.merge(df2, on = 'id', how='left')
    dfx1['merge'] = dfx1.apply(lambda x : rank_adjust2(x['list1'], x['list2'],), axis=1 )
    return dfx1[['id', 'merge']]

t0 = time.time()
result_0 = test_0(df1, df2)
print(time.time() -t0)
result_0['merge']

11.066330909729004


0        1000,1001,1999,1002,1003,1998,1004,1005,1997,1...
1        1000,1001,1999,1002,1003,1998,1004,1005,1997,1...
2        1000,1001,1999,1002,1003,1998,1004,1005,1997,1...
3        1000,1001,1999,1002,1003,1998,1004,1005,1997,1...
4        1000,1001,1999,1002,1003,1998,1004,1005,1997,1...
                               ...                        
19995    1000,1001,1999,1002,1003,1998,1004,1005,1997,1...
19996    1000,1001,1999,1002,1003,1998,1004,1005,1997,1...
19997    1000,1001,1999,1002,1003,1998,1004,1005,1997,1...
19998    1000,1001,1999,1002,1003,1998,1004,1005,1997,1...
19999    1000,1001,1999,1002,1003,1998,1004,1005,1997,1...
Name: merge, Length: 20000, dtype: object

In [2]:
### dask
from dask import dataframe as dd

dd1 = dd.from_pandas(df1, npartitions=2)
dd2 = dd.from_pandas(df2, npartitions=2)
#dd1 = dd1.set_index('id', sorted=True)
#dd2 = dd2.set_index('id', sorted=True)

#### algo
def test_1(dd1, dd2):
    def rank_adjust2(ll1, ll2):
        """ Re-rank elements of list1 using ranking of list2"""
        if isinstance(ll1, str): ll1 = ll1.split(",")
        if isinstance(ll2, str): ll2 = ll2.split(",")
        n1, n2 = len(ll1), len(ll2)
        if n2 < 1: return ll1

        ll2 = {x:i for i,x in enumerate( ll2 )  }

        # log(ll1) ; log(ll2)

        adjust, mrank = (1.0 * n1) / n2, n2
        rank3 = np.zeros(n1, dtype='float32')
        kk    = 2
        for rank1, sid in enumerate(ll1):
            rank2        = ll2.get(sid, mrank)
            rank3[rank1] = - rank_score2(rank1, rank2, adjust=adjust, kk= kk)

        # Id of ll1 sorted list
        v = [ll1[i] for i in np.argsort(rank3)]
        return ",".join( v)


    def rank_score2(rank1, rank2, adjust=1.0, kk=1.0):
        return 1.0 / (kk + rank1) + 1.0 / (kk + rank2 * adjust)

    dd3 = dd1.merge(dd2, on ='id', how='left')
    meta_df = ('list1', 'str')
    dd3['merge'] = dd3.apply(lambda x : rank_adjust2(x['list1'], x['list2'],), axis=1 , meta=meta_df)
    return dd3

task = test_1(dd1, dd2)
t0 = time.time()
result_1 = task.compute()
print(time.time() -t0)
result_1['merge']

11.889621257781982


0       1000,1001,1999,1002,1003,1998,1004,1005,1997,1...
1       1000,1001,1999,1002,1003,1998,1004,1005,1997,1...
2       1000,1001,1999,1002,1003,1998,1004,1005,1997,1...
3       1000,1001,1999,1002,1003,1998,1004,1005,1997,1...
4       1000,1001,1999,1002,1003,1998,1004,1005,1997,1...
                              ...                        
9885    1000,1001,1999,1002,1003,1998,1004,1005,1997,1...
9886    1000,1001,1999,1002,1003,1998,1004,1005,1997,1...
9887    1000,1001,1999,1002,1003,1998,1004,1005,1997,1...
9888    1000,1001,1999,1002,1003,1998,1004,1005,1997,1...
9889    1000,1001,1999,1002,1003,1998,1004,1005,1997,1...
Name: merge, Length: 20000, dtype: object

In [3]:
def is_same(dfa, dfb):
    if dfa.shape != dfb.shape:
        return False
    for i in range(dfa.shape[0]):
        if dfa.iloc[i] != dfb.iloc[i]:
            print(i,dfa.iloc[i],dfb.iloc[i])
            return False
    return True

In [4]:
is_same(result_0['merge'], result_1['merge'])

True